# Trabalho de Algoritmos e Estrutura de Dados
## Problema da Mochila

## Importando as dependências

In [2]:
from datetime import datetime
import pandas as pd

## Definindo o problema

In [3]:
# dados = {"valor": [60, 100, 120, 50],
#          "peso": [10, 20, 30, 50]}
# capacidade = 50

dados = {"valor": [10, 21, 50, 51],
         "peso": [2, 3, 5, 6]}
capacidade = 7

itens = pd.DataFrame(dados)
itens["valor_por_peso"] = itens.valor / itens.peso

## Algoritmos Exatos
---

### Força Bruta

In [3]:
from algoritmos_exatos import BruteForceKnapsack

In [7]:
inicio_processamento = datetime.now()

knapsack = BruteForceKnapsack(capacidade, itens)
print (f"Valor máximo que obtemos = {knapsack.max_1()}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

NameError: name 'BruteForceKnapsack' is not defined

### Programação Dinâmica

In [4]:
from algoritmos_exatos import DynamicProgrammingKnapsack

In [9]:
inicio_processamento = datetime.now()

knapsack = DynamicProgrammingKnapsack(capacidade, itens)
print (f"Valor máximo que obtemos = {knapsack.max_1()}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Selected weights are: 21 10 
None
Valor máximo que obtemos = 60
Tempo de processamento: 0:00:00.001001


### Branch and Bound

In [4]:
from algoritmos_exatos import BranchAndBoundKnapsack

In [5]:
inicio_processamento = datetime.now()

knapsack = BranchAndBoundKnapsack(capacidade, itens)
print (f"Valor máximo que obtemos = {knapsack.max()}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Valor máximo que obtemos = 60.0
Tempo de processamento: 0:00:00.023991


## Algoritmos Aproximados
---

### Algoritmo Guloso

In [9]:
from algoritmos_aproximados import GreedyKnapsack

In [10]:
inicio_processamento = datetime.now()

knapsack = GreedyKnapsack(capacidade, itens)
print (f"Valor máximo que obtemos = {knapsack.max()}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Valor máximo que obtemos = 50.0
Tempo de processamento: 0:00:00.001001


In [11]:
inicio_processamento = datetime.now()

knapsack = GreedyKnapsack(capacidade, itens, fractional=True)
print (f"Valor máximo que obtemos = {knapsack.max()}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Valor máximo que obtemos = 67.0
Tempo de processamento: 0:00:00.001001


([[2, 10], [5, 50]], 7, 60)